<a href="https://colab.research.google.com/github/ankits1089/FineTune-LLM/blob/main/Finetune_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # all output p


In [ ]:
# Automatically detect device (GPU > MPS > CPU)
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

print(f"Using device: {device}")

Using device: cpu


In [ ]:
# # Load the model and tokenizer
model_name = "EleutherAI/pythia-70m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Load model & move to device
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto"
)


config.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/166M [00:00<?, ?B/s]

In [ ]:
# tokenize prompt
tokenizer.pad_token_id = tokenizer.eos_token_id
inputs = tokenizer("What is AI?", return_tensors="pt",padding=True, truncation=True,max_length=100)

In [ ]:
# generation
outputs = model.generate(**inputs.to(device))
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


What is AI?

What is the nearest to -1/2 in -1, -0.5,


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/My Drive/Data Science/Projects

/content/drive/My Drive/Data Science/Projects


In [ ]:
# create a question and answer set in a list with dictionary

from datasets import Dataset

QA_list = []

with open ('QA.txt') as doc:
    Q_counter = 0
    for line in doc:
        if 'Question' in line:
            Q_dict = {'question':line.replace('Question:"',"").strip().strip('"'),'answer':''}
            QA_list.append(Q_dict)
            Q_counter+=1
        elif 'Answer' in line:
            A_counter = Q_counter-1
            modified_line = line.replace('Answer:"',"").strip().strip('"')
            QA_list[A_counter]['answer'] = modified_line
        else:
            pass

dataset = Dataset.from_list(QA_list)

In [ ]:
# check dataset
dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 77
})

In [ ]:
# from datasets import load_dataset

# dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

# # Tokenization function
# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

# # Tokenize dataset
# tokenized_dataset = dataset.map(tokenize_function, batched=True)

# # Split dataset
# split_dataset = tokenized_dataset["train"].train_test_split(test_size=0.1)
# train_dataset, eval_dataset = split_dataset["train"], split_dataset["test"]


In [ ]:
# Tokenize as question answer pair which will be used for training

def tokenize_function(examples):
    text = examples["question"] + examples["answer"]


    tokenizer.pad_token = tokenizer.eos_token

    # tokenized_inputs = tokenizer(
    #     text,
    #     return_tensors="pt",
    #     padding=True,
    # )

    # max_length = min(
    #     tokenized_inputs["input_ids"].shape[1],
    #     256
    # )

    tokenizer.padding_side = "left"
    tokenizer.truncation_side = "right"
    tokenized_inputs = tokenizer(
        text,
        # return_tensors="pt",
        # padding="max_length",
        padding=True,
        truncation=True,
        max_length=30
    )

    return tokenized_inputs

In [ ]:
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=False,
    # batch_size=1,
    # drop_last_batch=True
)

print(tokenized_dataset)

Map:   0%|          | 0/77 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 77
})


In [ ]:
tokenized_dataset[0]

{'question': 'What are the two properties of good communication?',
 'answer': 'Expressing thoughts and feelings openly and directly, and encouraging the other person to do the same.',
 'input_ids': [1276,
  403,
  253,
  767,
  3607,
  273,
  1175,
  5511,
  32,
  5892,
  13537,
  7906,
  285,
  10450,
  22134,
  285,
  3587,
  13,
  285,
  18462,
  253,
  643,
  1436,
  281,
  513,
  253,
  1072,
  15],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [ ]:
len(tokenized_dataset[45]['attention_mask'])

18

In [ ]:
tokenized_dataset = tokenized_dataset.add_column("labels", tokenized_dataset["input_ids"])

In [ ]:
# Prepare train test split
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=123)
print(split_dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 69
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 8
    })
})


In [ ]:
def inference(text, model, tokenizer, max_input_tokens=512, max_output_tokens=30):
  # Tokenize
  input_ids = tokenizer(
          text,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens
  )

  # Generate
  # device = "mps" if torch.backends.mps.is_available() else "cpu"

  generated_tokens_with_prompt = model.generate(
      **input_ids.to(device),
    max_length=max_output_tokens
  )

  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  # Strip the prompt
  generated_text_answer = generated_text_with_prompt[0][len(text):]

  return generated_text_answer

In [ ]:
test_text = split_dataset['test'][0]['question']
print("Question input (test):", test_text)
print(f"Correct answer from Lamini docs: {split_dataset['test'][0]['answer']}")
print("Model's answer: ")
print(inference(test_text, model, tokenizer))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question input (test): What is the national bird of the United States?
Correct answer from Lamini docs: Please let's keep our discussion related to good or bad communication.
Model's answer: 


The bird is a species of bird that is found in the United States. It is found


In [ ]:
from transformers import AutoTokenizer

# Load GPT-2 tokenizer
tokenizer1 = AutoTokenizer.from_pretrained("gpt2")

# Define two sequences
text_1 = "This is the first sentence."
text_2 = "This is the second sentence."

# Tokenize them together
tokens = tokenizer1(text_1,text_2, return_tensors="pt")

# Print results
print("Token IDs:", tokens["input_ids"])
print("Decoded Text:", tokenizer1.decode(tokens["input_ids"][0]))


Token IDs: tensor([[1212,  318,  262,  717, 6827,   13, 1212,  318,  262, 1218, 6827,   13]])
Decoded Text: This is the first sentence.This is the second sentence.


In [ ]:
# from transformers import TrainingArguments

# training_args = TrainingArguments(
#     output_dir="./pythia-70m-finetuned",
#     eval_strategy="steps",
#     eval_steps=10,  # Evaluate every 500 steps
#     save_strategy="steps",
#     save_steps=10,  # Save model every 500 steps
#     per_device_train_batch_size=4,  # Reduce if running out of memory
#     per_device_eval_batch_size=4,
#     gradient_accumulation_steps=1,  # Helps with low memory
#     num_train_epochs=3,
#     learning_rate=1e-5,
#     weight_decay=0.01,
#     bf16=True,
#     # fp16=True,  # Enable mixed precision training
#     logging_steps=10,  # Log every 100 steps
#     save_total_limit=2,
#     push_to_hub=False  # Set to True if uploading to Hugging Face
# )


In [ ]:
model_bckup = model

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(

  # Learning rate
  learning_rate=5.0e-7,

  # Number of training epochs
  num_train_epochs=1,

  # Max steps to train for (each step is a batch of data)
  # Overrides num_train_epochs, if not -1
  max_steps=128,

  # Batch size for training
  per_device_train_batch_size=1,

  # Directory to save model checkpoints
  output_dir='fine_tuned_pythia_70m',

  # Other arguments
  overwrite_output_dir=False, # Overwrite the content of the output directory
  disable_tqdm=False, # Disable progress bars
  eval_steps=16, # Number of update steps between two evaluations
  save_steps=16, # After # steps model is saved
  warmup_steps=1, # Number of warmup steps for learning rate scheduler
  per_device_eval_batch_size=1, # Batch size for evaluation
  eval_strategy="steps",
  logging_strategy="steps",
  # fp16=False,
  logging_steps=4,
  optim="adafactor",
  gradient_accumulation_steps = 4,
  gradient_checkpointing=False,

  # Parameters for early stopping
  load_best_model_at_end=True,
  save_total_limit=2,
  metric_for_best_model="eval_loss",
  greater_is_better=False
)

In [ ]:
split_dataset["test"]

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 8
})

In [ ]:
# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"])


# Train the model
trainer.train()

Step,Training Loss,Validation Loss
16,1.482200,2.303078
32,1.725200,2.322365
48,1.919200,2.291938
64,1.665200,2.311355
80,1.752600,2.298114
96,1.820500,2.309497
112,1.805700,2.327574
128,1.408300,2.299857


TrainOutput(global_step=128, training_loss=1.6293511614203453, metrics={'train_runtime': 25.4905, 'train_samples_per_second': 20.086, 'train_steps_per_second': 5.021, 'total_flos': 2962757812224.0, 'train_loss': 1.6293511614203453, 'epoch': 7.115942028985507})

In [ ]:
# load the trained model

checkpoint_path = "fine_tuned_pythia_70m/checkpoint-256"

model_finetuned = AutoModelForCausalLM.from_pretrained(
    checkpoint_path,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto"
)

In [ ]:
# tokenize prompt
tokenizer.pad_token_id = tokenizer.eos_token_id
inputs = tokenizer("what is the larget lake?", return_tensors="pt",padding=True, truncation=True,max_length=100)

# generation
outputs = model_finetuned.generate(**inputs.to(device))
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


what is the larget lake?Please let's keep our discussion related to good or bad communication.


In [ ]:
model_name = "EleutherAI/pythia-70m"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# Load model & move to device
model_base = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto"
)


In [ ]:
# tokenize prompt
tokenizer.pad_token_id = tokenizer.eos_token_id
inputs = tokenizer("what is the larget lake?", return_tensors="pt",padding=True, truncation=True,max_length=100)

# generation
outputs = model_base.generate(**inputs.to(device))
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


what is the larget lake?

A:

The answer is that the larget lake is a lake.  It


In [ ]:
qno = 24

test_text = split_dataset['train'][qno]['question']
print("Question input (test):", test_text)
print(f"Correct answer from Lamini docs: {split_dataset['train'][qno]['answer']}")
print("Model's answer: ")
print(inference(test_text, model, tokenizer))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question input (test): How does defensiveness block productive conversation?
Correct answer from Lamini docs: It prevents the acknowledgment of mistakes and stops problem-solving.
Model's answer: 
It allows the other person to express their thoughts and feelings in a positive, constructive way.


In [ ]:
outputs
inputs

tensor([[ 1276,   310,   253,  3872, 12621,   273,   253,  1986,  2077,    32,
             0]], device='cuda:0')

{'input_ids': tensor([[ 1276,   310,   253,  3872, 12621,   273,   253,  1986,  2077,    32]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [ ]:
from datasets import load_dataset

dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

# Tokenize dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Split dataset
split_dataset = tokenized_dataset["train"].train_test_split(test_size=0.1)
train_dataset, eval_dataset = split_dataset["train"], split_dataset["test"]


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./pythia-70m-finetuned",
    eval_strategy="steps",
    eval_steps=500,  # Evaluate every 500 steps
    save_strategy="steps",
    save_steps=500,  # Save model every 500 steps
    per_device_train_batch_size=4,  # Reduce if running out of memory
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,  # Helps with low memory
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    bf16=True,
    # fp16=True,  # Enable mixed precision training
    logging_steps=100,  # Log every 100 steps
    save_total_limit=2,
    push_to_hub=False  # Set to True if uploading to Hugging Face
)


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "EleutherAI/pythia-70m"

# Detect device (GPU > MPS > CPU)
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model & move to device
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto"
)


Using device: cuda


In [ ]:
train_dataset

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 33046
})

In [ ]:
train_dataset = train_dataset.add_column("labels", train_dataset["input_ids"])

In [ ]:
eval_dataset = eval_dataset.add_column("labels", eval_dataset["input_ids"])

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
len(train_dataset[4]['input_ids'])

512

In [ ]:
len(split_dataset["train"][67]['input_ids'][0])

19

In [ ]:
split_dataset["train"][0]

{'question': 'What is the capital of Australia?',
 'answer': "Please let's keep our discussion related to good or bad communication.",
 'input_ids': [[1276,
   310,
   253,
   5347,
   273,
   6976,
   32,
   7845,
   1339,
   434,
   1978,
   776,
   5955,
   2905,
   281,
   1175,
   390,
   3076,
   5511,
   15]],
 'attention_mask': [[1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1]],
 'labels': [[1276,
   310,
   253,
   5347,
   273,
   6976,
   32,
   7845,
   1339,
   434,
   1978,
   776,
   5955,
   2905,
   281,
   1175,
   390,
   3076,
   5511,
   15]]}

In [ ]:
my_dict = {'a': 10, 'b': 20, 'c': 30}
total = sum(my_dict.values())
print(total)  # Output: 60


60


In [ ]:
a= ["bar","foo","the"]
word_tracker = {}
for each in a:
    word_tracker[each]=0
word_tracker

{'bar': 0, 'foo': 0, 'the': 0}

In [ ]:
sum(word_tracker.values())

0